<a href="https://colab.research.google.com/github/Ananya-Joshi/CSTE_TA_Workbooks/blob/main/CSTE_Delphi_Walkthrough.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This walkthrough was initially developed for a guest class lecture at CMU and has been adapted for CSTE TA sessions!
Ananya Joshi

In [ ]:
# !pip install covidcast
import covidcast
import plotly.graph_objects as go
import pandas as pd
#REPLACE API KEY
covidcast.use_api_key("API KEY")
start_d = pd.to_datetime("05/01/2020")
end_d = pd.to_datetime("12/31/2021")
data_county = covidcast.signal("jhu-csse", "confirmed_incidence_num", start_d, end_d)
data_hrr = covidcast.signal("jhu-csse", "confirmed_incidence_num", start_d, end_d,"hrr")
data_state = covidcast.signal("jhu-csse", "confirmed_incidence_num", start_d, end_d,"state")
data_hhs = covidcast.signal("jhu-csse", "confirmed_incidence_num", start_d, end_d,"hhs")
data_nation = covidcast.signal("jhu-csse", "confirmed_incidence_num", start_d, end_d,"nation")
all_data = pd.concat([data_county, data_hrr, data_state, data_hhs, data_nation])

In [ ]:
from delphi_utils.geomap import GeoMapper
gmpr = GeoMapper()
pa_counties = gmpr.add_geocode(gmpr.add_geocode(data_county, 'fips', 'state_code', 'geo_value'), 'fips', 'hrr', 'geo_value').query('state_code=="42"')
hrr_list = pd.unique(pa_counties.hrr)
pa_hrrs = data_hrr.query('geo_value in @hrr_list')
pa_state = data_state.query('geo_value=="pa"')
all_data = pd.concat([pa_counties, pa_hrrs, pa_state])[['geo_value', 'geo_type', 'value', 'time_value']].drop_duplicates(subset=['geo_value', 'geo_type', 'time_value'])
all_data.to_csv("prototyping_data.csv")
cross_hierarchy_table = pa_counties[['geo_value', 'hrr']]
cross_hierarchy_table.columns = ['fips', 'hrr']
cross_hierarchy_table.to_csv("cross_hierarchy.csv")

#Part 1: Investigate selected data

In [ ]:
!pip install covidcast

In [ ]:
#restart
import covidcast

In [ ]:
import plotly.graph_objects as go
import pandas as pd

cross_hierarchy_table = pd.read_csv('https://github.com/Ananya-Joshi/AISG_Time_Series_Follow_Along/blob/main/cross_hierarchy.csv?raw=true', on_bad_lines='skip', header=0, index_col=0)
input_data = pd.read_csv('https://github.com/Ananya-Joshi/AISG_Time_Series_Follow_Along/blob/main/prototyping_data.csv?raw=true', on_bad_lines='skip', header=0, index_col=0)
input_data = input_data.astype({
                   'value': 'float',
                   })
input_data.time_value = pd.to_datetime(input_data.time_value)
unique_counties = pd.unique(input_data.query('geo_type=="county"').geo_value)
unique_hrr = pd.unique(input_data.query('geo_type=="hrr"').geo_value)
unique_state = pd.unique(input_data.query('geo_type=="state"').geo_value)
print(f"1. Data Structure : Unique Counties- {unique_counties.shape[0]} , Unique HRRs - {unique_hrr.shape[0]} , Unique States - {unique_state.shape[0]}\n")
total_days =  input_data.time_value.max() - input_data.time_value.min()
print(f"2. Date Ranges: {input_data.time_value.min()} - {input_data.time_value.max()} \n")

1. Data Structure : Unique Counties- 67 , Unique HRRs - 21 , Unique States - 1

2. Date Ranges: 2020-05-01 00:00:00 - 2021-12-31 00:00:00 



In [ ]:
#1a: Visualize 5 random streams
import plotly.graph_objects as go
import random
def plot_fn(data_stream):
  fig = go.Figure()
  fig.add_trace(go.Scatter(x = data_stream.time_value, y= data_stream.value)).update_layout(title = f"Sample #{i}, Geo:{geo}, Type:{input_data.query('geo_value==@geo').geo_type.iloc[0]}").show(rendere='colab')


print("3. Example of Data Streams \n")
random_set_geos = random.sample(list(pd.unique(input_data.geo_value)), 5)#.random(5)
for i, geo in enumerate(random_set_geos):
  data_stream = plot_fn(input_data.query('geo_value == @geo').sort_values(by='time_value'))


3. Example of Data Streams 



**1A:** What are some properties of the data that you notice?
+ Zoom into the data at different lenghts - what do you notice
+ Can you identify any similarities between the streams
+ What do the axes tell you?

In [ ]:
#1b: What type of summary statistics can help you better understand your data?

#Code Skeleton
for i, geo in enumerate(random_set_geos):
  data_stream = input_data.query('geo_value == @geo').sort_values(by='time_value')
  for window in [7]:#TODO: Fill in with the windows you want to see
      rolling_stream = data_stream[['time_value', 'value']].set_index('time_value').rolling(window).std() #TODO: what functions do you want to see
      plot_fn(rolling_stream.reset_index())

print(f"2a. Total Data Points: {input_data.shape[0]}")
print(f"2b. Data Missingness: { input_data[['geo_value','geo_type']].drop_duplicates().shape[0]*(total_days.days+1) - input_data.shape[0] }")
print(f"2c. Data Zeros: {(input_data == 0).sum().value}")

2a. Total Data Points: 54290
2b. Data Missingness: 0
2c. Data Zeros: 5146


**1B:**
1. How does varying the window change the plots? What is a good value for the window size?
2. What other summary statistics were worth inspecting?

#Part 2: Trying existing approaches

In [ ]:
#2a: Alerting Based Approach
from plotly.subplots import make_subplots

for i, geo in enumerate(random_set_geos):
  test_set_data = 10
  data_stream = input_data.query('geo_value == @geo').sort_values(by='time_value')
  training_data = data_stream.iloc[:100].copy()
  histogram_data = data_stream.iloc[100:-1*test_set_data].copy()
  test_data = data_stream.iloc[-1*test_set_data:].copy()
  fig = make_subplots(rows=1, cols=2, subplot_titles=(f'Sample #{i} Historgram',f"Geo:{geo}, Type:{input_data.query('geo_value==@geo').geo_type.iloc[0]}"))
  def outlier_processing(df):
    #Step 1: Forecast Values - AR =1
    df['pred'] = df.value.shift(1)
    #Step 2: calculate the difference between expected and observed values
    df['diff'] = abs(df['value'] - df['pred']) #TODO: What else can we do here?
    df = df.dropna()
    return df

  histogram_data = outlier_processing(histogram_data)
  #Step 3: compare to historical values
  hist_dist = histogram_data['diff']
  #Visualize the Histogram
  fig.add_trace(go.Histogram(x =hist_dist),
    row=1, col=1
)

  #Step 4: compare test statistics to historical quantiles
  test_data = outlier_processing(test_data)
  test_data['outlier_score'] = [abs(2*(sum(x <=hist_dist)/hist_dist.shape[0]-0.5)-1) for x in test_data['diff']]

  #Visualize the data and the outlier scores
  fig.add_trace(go.Scatter(x = data_stream.time_value, y= data_stream.value, name='opt', marker_color='black'
   ), row=1, col=2).add_trace(go.Scatter(x = data_stream.time_value[:-1], y= data_stream.value.shift(1),  marker_color='red', name='forecast'), row=1, col=2).show(rendere='colab') #


In [ ]:
#2c: Benchmarking for a single stream
def zscore_outliers(stream, window, thresh=3):
    roll = stream.rolling(window=window, min_periods=1, center=True)
    avg = roll.mean() #TODO: Can we replace this with a differnt function
    std = roll.std(ddof=0)
    z = stream.sub(avg).div(std).dropna()
    m = ~z.between(-thresh, thresh)
    return sum(m)

benchmarking_list = []
for thresh in [1, 3, 10]: #TODO
  for window in [7, 14, 21]: #TODO
    benchmarking_list.append({"num_outliers":zscore_outliers(data_stream.value, window, thresh = thresh),
                            "thresh": thresh,
                            "window": window})
benchmarking_df = pd.DataFrame(benchmarking_list)
#3D Plot
fig = go.Figure(data=[go.Scatter3d(x=benchmarking_df['thresh'], y=benchmarking_df['num_outliers'], z=benchmarking_df['window'], mode='markers')]).update_layout(scene=dict(
                    xaxis_title='Thresholds',
                    yaxis_title='# Outliers',
                    zaxis_title='Windows'
                    )).show(renderer='colab')
#2D plot for debugging
# fig = go.Figure(data=[go.Scatter(x=benchmarking_df['thresh'], y=benchmarking_df['num_outliers'], mode='markers')]).update_layout(xaxis_title='Thresholds',
#                     yaxis_title='# Outliers').show(renderer='colab')

In [ ]:
#2d: Benchmarking selected streams using binary outlier detection

#TODO: Identify which line numbers correspond to different steps of the outlier detection process
def zscore_outliers(stream, window, thresh=3):
    roll = stream.rolling(window=window, min_periods=1, center=True)
    avg = roll.mean()
    std = roll.std(ddof=0)
    z = stream.sub(avg).div(std).dropna()
    m = ~z.between(-thresh, thresh)
    return sum(m)

benchmarking_list = []
for thresh in [1, 3, 10]: #TODO
  for window in [7, 14, 21]: #TODO
    benchmarking_list.append({"num_outliers":zscore_outliers(data_stream.value, window, thresh = thresh),
                            "thresh": thresh,
                            "window": window})
benchmarking_df = pd.DataFrame(benchmarking_list)
#3D Plot
go.Figure(data=[go.Scatter3d(x=benchmarking_df['thresh'], y=benchmarking_df['num_outliers'], z=benchmarking_df['window'], mode='markers')]).update_layout(scene=dict(
                    xaxis_title='Thresholds',
                    yaxis_title='# Outliers',
                    zaxis_title='Windows'
                    )).show(renderer='colab')
#2D plot for debugging
# go.Figure(data=[go.Scatter(x=benchmarking_df['thresh'], y=benchmarking_df['num_outliers'], mode='markers')]).update_layout(xaxis_title='Thresholds',
#                     yaxis_title='# Outliers').show(renderer='colab')


In [ ]:
#To identify the number of outliers across ALL sterams, replace the num_outliers definition with the following:
sum(input_data.set_index(['time_value', 'geo_value'])['value'].groupby('geo_value').apply(lambda x: zscore_outliers(x, window, thresh)))

0

**2A/B**:
+ Compare and contrast the different algorithms among the metrics listed.
+ What other [metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) would you be interested in?
+ Did you change any of the parameters in the outlier detection methods?
+ How often would you need to change these parameters?
+ What types of outliers do these methods detect? What types do they miss?

#Part 3: Processing the Data


In [ ]:
#Part 3: Data processing stpes: Changepoint detection, remove outlier, Weekday smoothing
!pip install ruptures
import ruptures as rpt
from scipy.stats import nbinom, binom
import cvxpy as cp
import numpy as np
from cvxpy.error import SolverError
import warnings
from sklearn.linear_model import LinearRegression
import math



#3a: Identify & Visualize the changepoints

#Play with different methods and parameters from ruptures: https://centre-borelli.github.io/ruptures-docs/
bkpt_df = input_data.set_index(['geo_value','time_value'])['value'].unstack().T
#TODO: Investigate what this dataset looks like
ref_bktps = list(rpt.Pelt(model='rbf', min_size=28, jump=1).fit(bkpt_df.to_numpy()).predict(pen=10))
breakpoints = bkpt_df.index[ref_bktps[:-1]]
for i, geo in enumerate(random_set_geos):
  data_stream = input_data.query('geo_value == @geo').sort_values(by='time_value')
  #Visualize the data and the outlier scores
  fig = go.Figure().add_trace(go.Scatter(x = data_stream.time_value, y= data_stream.value
   ))
  for bkpt in breakpoints:
    fig.add_vline(bkpt)
  fig.show(rendere='colab')

In [ ]:
#Modified Weekday Convex Optimization Functions

def get_params(data, denominator_col, numerator_cols, date_col, scales, logger):
    r"""Fit weekday correction for each col in numerator_cols.

    Return a matrix of parameters: the entire vector of betas, for each time
    series column in the data.
    """
    tmp = data.reset_index()
    nums = tmp.groupby(date_col).sum()[numerator_cols]

    # Construct design matrix to have weekday indicator columns and then day
    # indicators.
    X = np.zeros((nums.shape[0], 6 + nums.shape[0]))
    not_sunday = np.where(nums.index.dayofweek != 6)[0]
    X[not_sunday, np.array(nums.index.dayofweek)[not_sunday]] = 1
    X[np.where(nums.index.dayofweek == 6)[0], :6] = -1
    X[:, 6:] = np.eye(X.shape[0])

    npnums = np.array(nums)
    params = np.zeros((nums.shape[1], X.shape[1]))

    npdenoms = None
    if denominator_col is not None:
        denoms = tmp.groupby(date_col).sum()[denominator_col]
        npdenoms = np.array(denoms)

    # Loop over the available numerator columns and smooth each separately.
    for i in range(nums.shape[1]):
        result = _fit(X, scales, npnums[:, i], npdenoms)
        if result is None:
            params[i, :] = np.pad(npnums[:, i], (6, 0), constant_values=(0,0))
        else:
            params[i,:] = result

    return params

def _fit(X, scales, npnums, npdenoms):
    r"""Correct a signal estimated as numerator/denominator for weekday effects.

    The ordinary estimate would be numerator_t/denominator_t for each time point
    t. Instead, model

    log(numerator_t/denominator_t) = alpha_{wd(t)} + phi_t

    where alpha is a vector of fixed effects for each weekday. For
    identifiability, we constrain \sum alpha_j = 0, and to enforce this we set
    Sunday's fixed effect to be the negative sum of the other weekdays.

    We estimate this as a penalized Poisson GLM problem with log link. We
    rewrite the problem as

    log(numerator_t) = alpha_{wd(t)} + phi_t + log(denominator_t)

    and set a design matrix X with one row per time point. The first six columns
    of X are weekday indicators; the remaining columns are the identity matrix,
    so that each time point gets a unique phi. Using this X, we write

    log(numerator_t) = X beta + log(denominator_t)

    Hence the first six entries of beta correspond to alpha, and the remaining
    entries to phi.

    The penalty is on the L1 norm of third differences of phi (so the third
    differences of the corresponding columns of bƒeta), to enforce smoothness.
    Third differences ensure smoothness without removing peaks or valleys.

    Objective function is negative mean Poisson log likelihood plus penalty:

    ll = (numerator * (X*b + log(denominator)) - sum(exp(X*b) + log(denominator)))
            / num_days
    """
    b = cp.Variable((X.shape[1]))

    lmbda = cp.Parameter(nonneg=True)
    lmbda.value = 10  # Hard-coded for now, seems robust to changes

    ll = (
                  cp.matmul(npnums, cp.matmul(X, b)) -
                  cp.sum(cp.exp(cp.matmul(X, b)))
          ) / X.shape[0]

    if npdenoms is not None:
        ll = (
                      cp.matmul(npnums, cp.matmul(X, b) + np.log(npdenoms)) -
                      cp.sum(cp.exp(cp.matmul(X, b) + np.log(npdenoms)))
              ) / X.shape[0]
    # L-1 Norm of third differences, rewards smoothness
    penalty = lmbda * cp.norm(cp.diff(b[6:], 3), 1) / (X.shape[0] - 2)
    for scale in scales:
        try:
            prob = cp.Problem(cp.Minimize((-ll + lmbda * penalty) / scale))
            with  warnings.catch_warnings(record=True) as w:
              _ = prob.solve()
              for warning in w:
                if 'inaccurate' in str(warning.message):
                  print(warning)
                  continue
            return b.value
        except SolverError:
            # If the magnitude of the objective function is too large, an error is
            # thrown; Rescale the objective function by going through loop
            continue
    return None


In [ ]:
!pip install delphi_utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.8/357.8 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.3/219.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.2/55.2 kB 5.9 MB/s et

In [ ]:
import covidcast
from delphi_utils import Weekday
# Basic Approach Per Segment

prior_bkpt = 0
full_processed_df = []
for bkpt in ref_bktps:
  data_slice = bkpt_df.iloc[prior_bkpt:bkpt, :]
  smoothened_data = []

  for geo in data_slice.columns:
    #3b global outlier removal
    data_col = pd.DataFrame(data_slice[geo]).copy()
    data_col['score'] = (data_col - data_col.mean())/data_col.std()
    data_col['weekday'] = data_col.index.day_of_week
    impute_values = data_col.iloc[:, [0, 2]].groupby('weekday').median()
    high_indices = data_col.query('score>5').index
    for idx in high_indices:
      data_col.loc[idx, 'value'] = impute_values.loc[data_col.loc[idx, 'weekday']].values[0]

    #3c simplified smoothing
    scale = [1, 1e5, 1e10, 1e15]
    additive_factor = 2
    weekday_params = get_params((data_col+additive_factor).reset_index(), None, [data_col.columns[0]], 'time_value', scale, False)
    data_col_smooth = (Weekday.calc_adjustment(weekday_params, \
                                  (data_col+additive_factor).reset_index(),  [data_col.columns[0]], 'time_value').set_index('time_value')-additive_factor).clip(0)
    smoothened_data.append(data_col_smooth.iloc[:, 0])
  full_processed_df.append(pd.concat(smoothened_data, axis=1))
full_processed_df = pd.concat(full_processed_df, axis=0)


{message : UserWarning('Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.'), category : 'UserWarning', filename : '/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py', lineno : 1387, line : None}
{message : UserWarning('Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.'), category : 'UserWarning', filename : '/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py', lineno : 1387, line : None}
{message : UserWarning('Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.'), category : 'UserWarning', filename : '/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py', lineno : 1387, line : None}
{message : UserWarning('Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for

In [ ]:
#3b.Comparing Orginal and Processed df

for i, geo in enumerate(random_set_geos):
  data_stream = input_data.query('geo_value == @geo').sort_values(by='time_value')
  data_stream_smth = full_processed_df[geo].sort_index()
  #Visualize the data and the outlier scores
  fig = go.Figure().add_trace(go.Scatter(x = data_stream.time_value, y= data_stream.value
   )).add_trace(go.Scatter(x = data_stream_smth.index, y= data_stream_smth
   )).show(rendere='colab')

In [ ]:
diff_metric = (((bkpt_df - full_processed_df)**2).sum().sum()/(bkpt_df.shape[0]*bkpt_df.shape[1]))**0.5

In [ ]:
diff_metric

149.82712130873887